## Phishing Site Detection

In the Pre processing of the Data the attributes are coded into following format for application of KNN Classifier 

 	SFH's type is nominal, range is ('1', '-1', '0')
 	popUpWidnow's type is nominal, range is ('-1', '0', '1')
 	SSLfinal_State's type is nominal, range is ('1', '-1', '0')
 	Request_URL's type is nominal, range is ('-1', '0', '1')
 	URL_of_Anchor's type is nominal, range is ('-1', '0', '1')
 	web_traffic's type is nominal, range is ('1', '0', '-1')
 	URL_Length's type is nominal, range is ('1', '-1', '0')
 	age_of_domain's type is nominal, range is ('1', '-1')
 	having_IP_Address's type is nominal, range is ('0', '1')
 	Result's type is nominal, range is ('0', '1', '-1'))
    
    The  data coding is as follows:
    "1"  - Legitimate Website
    "0"  - Suspisious
    "-1" - Malicious Website

In [1]:
your_local_path = 'C:\\Users\\jatsharm\\DataScience\\MachineLearning\dataset\\'

In [2]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from scipy.io.arff import loadarff
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
phishing_data_raw = pd.read_csv(your_local_path+"PhishingData.csv")

In [4]:
phishing_data_raw

(array([(b'1', b'-1', b'1', b'-1', b'-1', b'1', b'1', b'1', b'0', b'0'),
        (b'-1', b'-1', b'-1', b'-1', b'-1', b'0', b'1', b'1', b'1', b'1'),
        (b'1', b'-1', b'0', b'0', b'-1', b'0', b'-1', b'1', b'0', b'1'),
        ...,
        (b'-1', b'0', b'-1', b'-1', b'-1', b'0', b'-1', b'-1', b'0', b'1'),
        (b'0', b'0', b'1', b'0', b'0', b'0', b'-1', b'1', b'0', b'1'),
        (b'1', b'0', b'1', b'1', b'1', b'0', b'-1', b'-1', b'0', b'-1')], 
       dtype=[('SFH', 'S2'), ('popUpWidnow', 'S2'), ('SSLfinal_State', 'S2'), ('Request_URL', 'S2'), ('URL_of_Anchor', 'S2'), ('web_traffic', 'S2'), ('URL_Length', 'S2'), ('age_of_domain', 'S2'), ('having_IP_Address', 'S1'), ('Result', 'S2')]),
 Dataset: 'Phishing-weka.filters.supervised.attribute.AttributeSelection-Eweka.attributeSelection.InfoGainAttributeEval-Sweka.attributeSelection.Ranker
 	SFH's type is nominal, range is ('1', '-1', '0')
 	popUpWidnow's type is nominal, range is ('-1', '0', '1')
 	SSLfinal_State's type is nominal, r

In [5]:
# Selection of the Data and converting data into numpy format for flexibility in cleaning
phising_data_array = np.array(phishing_data_raw[0])
phising_data_array

array([(b'1', b'-1', b'1', b'-1', b'-1', b'1', b'1', b'1', b'0', b'0'),
       (b'-1', b'-1', b'-1', b'-1', b'-1', b'0', b'1', b'1', b'1', b'1'),
       (b'1', b'-1', b'0', b'0', b'-1', b'0', b'-1', b'1', b'0', b'1'),
       ...,
       (b'-1', b'0', b'-1', b'-1', b'-1', b'0', b'-1', b'-1', b'0', b'1'),
       (b'0', b'0', b'1', b'0', b'0', b'0', b'-1', b'1', b'0', b'1'),
       (b'1', b'0', b'1', b'1', b'1', b'0', b'-1', b'-1', b'0', b'-1')], 
      dtype=[('SFH', 'S2'), ('popUpWidnow', 'S2'), ('SSLfinal_State', 'S2'), ('Request_URL', 'S2'), ('URL_of_Anchor', 'S2'), ('web_traffic', 'S2'), ('URL_Length', 'S2'), ('age_of_domain', 'S2'), ('having_IP_Address', 'S1'), ('Result', 'S2')])

In [6]:
# Converting the numpy array into Pandas data frame  and casting the coloumns to numeric type
phising_data_frame = pd.DataFrame(phising_data_array).apply(pd.to_numeric)
phising_data_frame[0:10]

,SFH,popUpWidnow,SSLfinal_State,Request_URL,URL_of_Anchor,web_traffic,URL_Length,age_of_domain,having_IP_Address,Result
0,1,-1,1,-1,-1,1,1,1,0,0
1,-1,-1,-1,-1,-1,0,1,1,1,1
2,1,-1,0,0,-1,0,-1,1,0,1
3,1,0,1,-1,-1,0,1,1,0,0
4,-1,-1,1,-1,0,0,-1,1,0,1
5,-1,-1,1,-1,-1,1,0,-1,0,1
6,1,-1,0,1,-1,0,0,1,0,-1
7,1,0,1,1,0,0,0,1,1,-1
8,-1,-1,0,-1,-1,-1,-1,1,0,0
9,-1,0,-1,-1,1,1,0,-1,0,1


In [7]:
col_names = list(phising_data_frame)# Geeting Column names of the pandas data frame
print("Coloumn_names:" + ", ".join(col_names))
# The scikit learn package takes data in form of labels and predictors
# Extracting Result coloum to pass as target coloum
labels = phising_data_frame["Result"].values
print(labels)

Coloumn_names:SFH, popUpWidnow, SSLfinal_State, Request_URL, URL_of_Anchor, web_traffic, URL_Length, age_of_domain, having_IP_Address, Result
[ 0  1  1 ...,  1  1 -1]


In [8]:
predictor_cols = col_names[0:len(col_names)-2]# selecting the predictor column names
print("Predictor_Coloumns:" + ", ".join(predictor_cols))
#extracting Predictor points 
predictors = phising_data_frame[predictor_cols].values
predictors

Predictor_Coloumns:SFH, popUpWidnow, SSLfinal_State, Request_URL, URL_of_Anchor, web_traffic, URL_Length, age_of_domain


array([[ 1, -1,  1, ...,  1,  1,  1],
       [-1, -1, -1, ...,  0,  1,  1],
       [ 1, -1,  0, ...,  0, -1,  1],
       ..., 
       [-1,  0, -1, ...,  0, -1, -1],
       [ 0,  0,  1, ...,  0, -1,  1],
       [ 1,  0,  1, ...,  0, -1, -1]], dtype=int64)

In [9]:
# Defining The parameters in KNN Classifier
clf_knn = KNeighborsClassifier(
    n_neighbors=10,
    weights='distance'
    )
clf_knn = clf_knn.fit(predictors,labels)

In [10]:
# Computing the Cross validation score with 5-fold cross validation
score_knn = cross_val_score(clf_knn, predictors, labels, cv=5)
print("Cross Validation score : " + str(score_knn))
print("Cross Validation Mean score : " + str(score_knn.mean()))

Cross Validation score : [ 0.84926471  0.88602941  0.88560886  0.89219331  0.89591078]
Cross Validation Mean score : 0.881801412591


In [11]:
# Add-on
# After finalizing the value of 'k' post Cross validation we can apply ML model as below.

col_names_ml = list(phising_data_frame)# Geeting Column names of the pandas data frame
print("Coloumn_names:" + ", ".join(col_names_ml))
# The scikit learn package takes data in form of labels and predictors
# Extracting Result coloum to pass as target coloum
labels_ml = phising_data_frame["Result"].values
print(labels_ml)



predictor_cols_ml = col_names_ml[0:len(col_names_ml)-2]# selecting the predictor column names
print("Predictor_Coloumns:" + ", ".join(predictor_cols_ml))
#extracting Predictor points 
predictors_ml = phising_data_frame[predictor_cols_ml].values
print(predictors_ml)


# split into train and test
X_train, X_test, y_train, y_test = train_test_split(predictors_ml, labels_ml, test_size=0.3, random_state=42)

# Defining The parameters in KNN Classifier
clf_knn_ml = KNeighborsClassifier(n_neighbors=10,weights='distance')
clf_knn_ml = clf_knn.fit(X_train,y_train)

pred = clf_knn_ml.predict(X_test)
target_names = ['-1','0', '1']
print(classification_report(y_test,pred,target_names=target_names))
confusion_matrix(y_test,pred)

Coloumn_names:SFH, popUpWidnow, SSLfinal_State, Request_URL, URL_of_Anchor, web_traffic, URL_Length, age_of_domain, having_IP_Address, Result
[ 0  1  1 ...,  1  1 -1]
Predictor_Coloumns:SFH, popUpWidnow, SSLfinal_State, Request_URL, URL_of_Anchor, web_traffic, URL_Length, age_of_domain
[[ 1 -1  1 ...,  1  1  1]
 [-1 -1 -1 ...,  0  1  1]
 [ 1 -1  0 ...,  0 -1  1]
 ..., 
 [-1  0 -1 ...,  0 -1 -1]
 [ 0  0  1 ...,  0 -1  1]
 [ 1  0  1 ...,  0 -1 -1]]
             precision    recall  f1-score   support

         -1       0.92      0.91      0.91       203
          0       0.83      0.83      0.83        35
          1       0.87      0.89      0.88       168

avg / total       0.89      0.89      0.89       406



array([[184,   3,  16],
       [  0,  29,   6],
       [ 16,   3, 149]])

In [12]:
print(accuracy_score(y_test,pred))

0.891625615764
